In [1]:
import os 


In [2]:
!pwd



/home/snow/Pictures/Kidney-Disease-Classification-Deep-learning-project/research


In [3]:
os.chdir("../")

In [ ]:
export MLFLOW_TRACKING_URI=https://dagshub.com/pavan-3000/Kidney-Disease-Classification-Deep-learning-project.mlflow 
export MLFLOW_TRACKING_USERNAME=pavan-3000 
export MLFLOW_TRACKING_PASSWORD=857d6e6289483a29a4f4c565648addd2cfd6d7d9 

In [ ]:
MLFLOW_TRACKING_URI=https://dagshub.com/pavan-3000/Kidney-Disease-Classification-Deep-learning-project.mlflow \
MLFLOW_TRACKING_USERNAME=pavan-3000 \
MLFLOW_TRACKING_PASSWORD=857d6e6289483a29a4f4c565648addd2cfd6d7d9 \
python script.py

In [20]:
os.environ["MLFLOW_TRACKING_URI"]= "https://dagshub.com/pavan-3000/Kidney-Disease-Classification-Deep-learning-project.mlflow "
os.environ["MLFLOW_TRACKING_USERNAME"]= "pavan-3000" 
os.environ["MLFLOW_TRACKING_PASSWORD"]= "857d6e6289483a29a4f4c565648addd2cfd6d7d9 "


In [5]:
import tensorflow as tf

2024-04-07 10:10:40.193364: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-07 10:10:41.140676: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-07 10:10:47.885185: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 10:10:47.885357: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 10:10:49.173753: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class EvaluteConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_url: str
    params_image_size: list
    params_batch_size: int

In [8]:
from src.project.constants import *
from src.project.utils.common import read_yaml,create_directory

In [14]:
class configmanager:
    def __init__(
        self,
        config_path = CONFIG_FILE_PATH,
        params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_directory([self.config.artifacts_root])
        
    def EvalManager(self) -> EvaluteConfig:
        eval_config = EvaluteConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_url="https://dagshub.com/pavan-3000/Kidney-Disease-Classification-Deep-learning-project.mlflow",
            all_params= self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
            
            
        )
        return eval_config

In [10]:
import mlflow
import mlflow.keras


In [11]:
from src.project.utils.common import save_json
from urllib.parse import urlparse

In [18]:
class Evaluation:
    def __init__(self,config:EvaluteConfig):
        self.config = config
        
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    
    def evaluation(self):
        self.model  = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        score = {"loss": self.save_score[0],"accuracy": self.score[1]}
        save_json(Path=Path("scores.json"),data = score)
        
        
    def log_into_mlfow(self):
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0],"accuracy": self.score[1]}
            )
            
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model,"model",registered_model_name = "VGG16Model")
            else:
                mlflow.keras.log_model(self.model,"model")
                
    
    

In [23]:
config = configmanager()
config = config.EvalManager()
evalute = Evaluation(config=config)
evalute.evaluation()
evalute.log_into_mlfow()

Found 139 images belonging to 2 classes.
9/9 [==============================] - 128s 4s/step - loss: 1.3209 - accuracy: 0.9137


MlflowException: API request to endpoint /api/2.0/mlflow/runs/create failed with error code 401 != 200. Response body: '=============== ATTENTION! ===============

To use authentication, you must first: 
    Get your default access token from: https://dagshub.com/user/settings/tokens
    OR 
    Set a password: https://dagshub.com/user/settings/password 
=========================================='

In [ ]:
857d6e6289483a29a4f4c565648addd2cfd6d7d9